In [2]:
# Denpendencies

import requests
%matplotlib inline
from pathlib import Path
import pandas as pd
import scipy.stats as stats
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import hvplot.pandas

import warnings
warnings.filterwarnings('ignore')

# Import the API keys
from API_Keys import API_KEY
from API_Keys import API_SECRET
from API_Keys import geoapify_key


ModuleNotFoundError: No module named 'API_Keys'

In [ ]:
# Pull in Petfinder API credentials

def get_auth_token():
    """Authenticate with the Petfinder API and return the access token."""
    url = "https://api.petfinder.com/v2/oauth2/token"
    data = {
        "grant_type": "client_credentials",
        "client_id": API_KEY,
        "client_secret": API_SECRET,
    }
    response = requests.post(url, data=data)
    response.raise_for_status()  # Ensure the request was successful
    return response.json()["access_token"]

def fetch_adoptable_pets(cities, token):
    """Fetch adoptable pets (cats and dogs) information for specified cities."""
    url = "https://api.petfinder.com/v2/animals"
    headers = {"Authorization": f"Bearer {token}"}
    pets_data = []

    for city in cities:
        for animal_type in ['Cat', 'Dog']:
            params = {
                "type": animal_type,
                "location": city,
                "limit": 100,
            }
            page = 1  # Start from the first page

            while True:
                params["page"] = page
                response = requests.get(url, headers=headers, params=params)
                response.raise_for_status()
                data = response.json()

                for animal in data["animals"]:
                    pets_data.append({
                        "type": animal["type"],
                        "breed": animal["breeds"]["primary"],
                        "location": animal["contact"]["address"]["city"],
                        "color": animal["colors"]["primary"] if animal["colors"]["primary"] else "N/A",
                        "size": animal["size"],
                        "date_posted": animal["published_at"],
                        "name": animal["name"],
                        "status": animal["status"],
                        "gender": animal["gender"]
                    })

                if page >= data["pagination"]["total_pages"]:
                    break
                page += 1

    return pd.DataFrame(pets_data)

# Cities to fetch data for
cities = ["New York City, NY", "Los Angeles, CA", "Chicago, IL", "Houston, TX", "Phoenix, AZ"]

# Authenticate
token = get_auth_token()

# Fetch pets data
pets_df = fetch_adoptable_pets(cities, token)

# Display the DataFrame
print(pets_df.head())

In [ ]:
# Create a file path
adoption_data_df = Path("C:/Users/14407/Documents/STAK_Project_One/Resources/adoption.csv", index_label="Animal_ID")


In [ ]:
#Read saved data
adoption_data_df = pd.read_csv("C:/Users/14407/Documents/STAK_Project_One/Resources/adoption.csv", index_col="Animal_ID")

#Display sample data
adoption_data_df.head()